# Raw Code
The following cell includes a class that stores the puzzle, and has a method that solves itself. This method uses a few heuristic real-life strategies (clearing notes, looking to see if a row/col/box has only one spot for a number, obvious doubles) and a brute force real-life strategy, Bowman's Bingo, which basically tests an assumption.

This code does nothing on its own, but make sure to run it before the tests/examples!

In [12]:
from IPython.display import clear_output
from time import sleep


class Cell:

    def __init__(self, value=0):
        self.value = value
        self.hasClearedRelatives = False
        if value == 0:
            self.notes = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        else:
            self.notes = None

    def __str__(self):
        return str(self.value)

    def setValue(self, num):
        self.value = num
        self.notes = None

    def clearNote(self, num):
        """
        Clears num from notes, and changes value to last remaining note if applicable.
        args:
        num -- the number to clear from notes
        returns:
        False if no change was made (num was already cleared)
        True if num was successfully cleared
        """
        if self.notes is None or num not in self.notes:
            return False

        self.notes.remove(num)
        if len(self.notes) == 1:
            self.value = self.notes[0]
            self.notes = None
        return True


class SudokuPuzzle:

    def __init__(
        self,
        nums="000000000000000000000000000000000000000000000000000000000000000000000000000000000"
    ):
        """
        initializes the puzzle
        args:
        nums -- a string that represents the puzzle's starting values. 0 means unfilled. Left to right then top to bottom
        """

        self.cells = [[Cell() for x in range(9)] for y in range(9)]

        for i in range(81):
            if nums[i]:
                self.cells[i // 9][i % 9] = Cell(int(nums[i]))

    def __str__(self):
        output = ""
        row = 0
        for line in self.cells:
            col = 0
            for cell in line:
                output += cell.__str__()
                col += 1
                if col == 3 or col == 6:
                    output += '|'
            output += '\n'
            row += 1
            if row == 3 or row == 6:
                output += '-----------'
                output += '\n'
        return output

    def cellsToString(self):
        output = ""
        for line in self.cells:
            for cell in line:
                output += str(cell.value)
        return output

    def clearNotes(self, coords=None):
        """
        Clears all notes that a particular cell disallows. So if a cell is a 7, it clears all 7s from the notes of all cells in its row, column, and box. 
        If cellCoords is None, then applies this method to all cells.
        args:
        coords -- a tuple of ints, (row,col), signifying the cell to clear relatives of OR None. If None, then applies this method to all cells.
        returns:
        True if this made a change
        False otherwise
        """

        change = False
        if coords is None:
            for row in range(9):
                for col in range(9):
                    if self.clearNotes((row, col)):
                        change = True
            return change

        if self.cells[coords[0]][coords[1]].value != 0 and not self.cells[
                coords[0]][coords[1]].hasClearedRelatives:
            # Clear row
            for cell in self.cells[coords[0]]:
                if cell.clearNote(self.cells[coords[0]][coords[1]].value):
                    change = True
            # Clear column
            for row in range(9):
                if self.cells[row][coords[1]].clearNote(
                        self.cells[coords[0]][coords[1]].value):
                    change = True
            # Clear box
            for x in range(3):
                for y in range(3):
                    # we use integer/floor division then re-multiplication to round down to the nearest 3
                    if self.cells[coords[0] // 3 * 3 +
                                  x][coords[1] // 3 * 3 + y].clearNote(
                                      self.cells[coords[0]][coords[1]].value):
                        change = True
            self.cells[coords[0]][coords[1]].hasClearedRelatives = True
        return change

    def findLoneSpot(self, num=None):
        """
        Checks to see if there's a row, column, or box that only has one spot that could have this number
        args:
        num -- the num to search for. If None, then reruns this with each num
        returns:
        True if a change was made
        False if not
        """
        change = False

        if num is None:
            for i in range(1, 10):
                if self.findLoneSpot(i):
                    change = True
        else:
            for line in self.cells:
                numCount = 0
                alreadyHasNum = False
                for cell in line:
                    if cell.value == num:
                        alreadyHasNum = True
                    if cell.notes is not None and num in cell.notes:
                        numCount += 1
                if not alreadyHasNum and numCount == 1:
                    for cell in line:
                        if cell.notes is not None and num in cell.notes:
                            cell.setValue(num)
                            change = True
            for col in range(9):
                numCount = 0
                alreadyHasNum = False
                for row in range(9):
                    if self.cells[row][col].value == num:
                        alreadyHasNum = True
                    if self.cells[row][
                            col].notes is not None and num in self.cells[row][
                                col].notes:
                        numCount += 1
                if not alreadyHasNum and numCount == 1:
                    for row in range(9):
                        if self.cells[row][
                                col].notes is not None and num in self.cells[
                                    row][col].notes:
                            self.cells[row][col].setValue(num)
                            change = True
            for boxRow in range(0, 9, 3):
                for boxCol in range(0, 9, 3):
                    numCount = 0
                    alreadyHasNum = False
                    for addRow in range(3):
                        for addCol in range(3):
                            if self.cells[boxRow +
                                          addRow][boxCol +
                                                  addCol].value == num:
                                alreadyHasNum = True
                            if self.cells[boxRow + addRow][
                                    boxCol +
                                    addCol].notes is not None and num in self.cells[
                                        boxRow + addRow][boxCol +
                                                         addCol].notes:
                                numCount += 1
                    if not alreadyHasNum and numCount == 1:
                        for addRow in range(3):
                            for addCol in range(3):
                                if self.cells[boxRow + addRow][
                                        boxCol +
                                        addCol].notes is not None and num in self.cells[
                                            boxRow + addRow][boxCol +
                                                             addCol].notes:
                                    self.cells[boxRow +
                                               addRow][boxCol +
                                                       addCol].setValue(num)
                                    change = True
        return change

    def obviousDoubles(self):
        """
        Checks for obvious doubles. That is when two cells in the same row, column or box have the same pair of possible numbers.
        When an obvious double is found, those two numbers are cleared from the row/column/box, respectively.
        """
        change = False
        for row in range(9):
            for col in range(9):
                cell = self.cells[row][col]
                if cell.notes is not None and len(cell.notes) == 2:
                    # Check row
                    for checkedCol in range(9):
                        if checkedCol != col:
                            checkedCell = self.cells[row][checkedCol]
                            # if checkedCell has the same number-notes
                            if checkedCell.notes is not None and len(
                                    checkedCell.notes) == 2 and cell.notes[
                                        0] in checkedCell.notes and cell.notes[
                                            1] in checkedCell.notes:
                                # clear the row of those number-notes
                                for clearedCol in range(9):
                                    if clearedCol != col and clearedCol != checkedCol:
                                        if self.cells[row][
                                                clearedCol].clearNote(
                                                    cell.notes[0]):
                                            change = True
                                        if self.cells[row][
                                                clearedCol].clearNote(
                                                    cell.notes[1]):
                                            change = True
                    # Check column
                    for checkedRow in range(9):
                        if checkedRow != row:
                            checkedCell = self.cells[checkedRow][col]
                            # if checkedCell has the same number-notes
                            if checkedCell.notes is not None and len(
                                    checkedCell.notes) == 2 and cell.notes[
                                        0] in checkedCell.notes and cell.notes[
                                            1] in checkedCell.notes:
                                for clearedRow in range(9):
                                    # clear the column of those number-notes
                                    if clearedRow != row and clearedRow != checkedRow:
                                        if self.cells[clearedRow][
                                                col].clearNote(cell.notes[0]):
                                            change = True
                                        if self.cells[clearedRow][
                                                col].clearNote(cell.notes[1]):
                                            change = True
                    # Clear box
                    for addRow in range(3):
                        for addCol in range(3):
                            # we use integer/floor division then re-multiplication to round down to the nearest 3
                            if row // 3 * 3 + addRow != row and col // 3 * 3 + addCol != col:
                                checkedCell = self.cells[row // 3 * 3 +
                                                         addRow][col // 3 * 3 +
                                                                 addCol]
                                # if checkedCell has the same number-notes
                                if checkedCell.notes is not None and len(
                                        checkedCell.notes
                                ) == 2 and cell.notes[
                                        0] in checkedCell.notes and cell.notes[
                                            1] in checkedCell.notes:
                                    # clear the box of those number-notes
                                    for clearedRow in range(
                                            row // 3 * 3, row // 3 * 3 + 3):
                                        for clearedCol in range(
                                                col // 3 * 3,
                                                col // 3 * 3 + 3):
                                            if (
                                                    clearedRow != row
                                                    or clearedCol != col
                                            ) and (clearedRow != row // 3 * 3 +
                                                   addRow or clearedCol
                                                   != col // 3 * 3 + addCol):
                                                if self.cells[clearedRow][
                                                        clearedCol].clearNote(
                                                            cell.notes[0]):
                                                    change = True
                                                if self.cells[clearedRow][
                                                        clearedCol].clearNote(
                                                            cell.notes[1]):
                                                    change = True
        return change

    def bowmansBingo(self):
        """
        Tries out a candidate for a cell (taken from its number-notes) by creating an alternate timeline where the cell's number is set to the candidate. Either the candidate is confirmed not to work 
        (in which case it is removed from the notes of that cell) or it works (in which case, all cells are copied from the alternate timeline puzzle to this one).
        This can be considered the brute force option, as it can be called recursively to eventually try out full puzzle solutions.
        """
        for row in range(9):
            for col in range(9):
                if self.cells[row][col].value == 0:
                    newInput = list(self.cellsToString())
                    newInput[row * 9 + col] = str(
                        self.cells[row][col].notes[0])[0]
                    newPuzzle = SudokuPuzzle("".join(newInput))
                    if newPuzzle.solve(printUpdates=False):
                        self.cells = newPuzzle.cells
                    else:
                        self.cells[row][col].clearNote(
                            self.cells[row][col].notes[0])

    def validate(self):
        """
        Checks if this puzzle is valid, as in there's no conflicts between cell numbers (e.g. two 4's in one row) and no cells have no value and no possible value
        returns:
        True if its still valid
        False if there's a conflict between two cells
        """
        for row in range(9):
            for col in range(9):
                if self.cells[row][col].value == 0:
                    if (self.cells[row][col].notes is None
                            or len(self.cells[row][col].notes) == 0):
                        return False
                else:
                    # Check row
                    for checkedCol in range(9):
                        if checkedCol != col and self.cells[row][
                                checkedCol].value == self.cells[row][col].value:
                            return False
                    # Check column
                    for checkedRow in range(9):
                        if checkedRow != row and self.cells[checkedRow][
                                col].value == self.cells[row][col].value:
                            return False
                    # Clear box
                    for x in range(3):
                        for y in range(3):
                            # we use integer/floor division then re-multiplication to round down to the nearest 3
                            if (row // 3 * 3 + x != row or col // 3 * 3 + y
                                    != col) and self.cells[row // 3 * 3 + x][
                                        col // 3 * 3 +
                                        y].value == self.cells[row][col].value:
                                return False
        return True

    def filled(self):
        """
        Checks if this puzzle is filled.
        returns:
        True if filled
        False if there's any empty cells
        """
        for line in self.cells:
            for cell in line:
                if cell.value == 0:
                    return False
        return True

    def solve(self, printUpdates=True, sleepTime=0):
        """
        Solves this puzzle
        returns:
        True if it solves it completely.
        False if it proves to be an impossible puzzle.
        """

        numIterations = 0
        while True:
            if not self.validate():
                return False
            if printUpdates:
                if sleepTime > 0:
                    sleep(sleepTime)
                clear_output(wait=True)
                print("After " + str(numIterations) + " iterations:")
                print(self)
            numIterations += 1
            if self.filled():
                return True
            if not self.clearNotes():
                if not self.findLoneSpot():
                    if not self.obviousDoubles():
                        self.bowmansBingo()

# Example Walk-through
This next code block grabs a random puzzle from a bank of "diabolical" (hardest difficulty) puzzles from the [Sudoku Exchange Puzzlebank](https://github.com/grantm/sudoku-exchange-puzzle-bank), a bunch of solve-able Sudoku puzzles in the public domain.
It then runs it through the above code block, with it set to only progress every 1.1 seconds (I would go faster but cocalc doesn't like updating more than every second). Each iteration of progress is the solver applying one method (note-clearing, obvious doubles, etc.) to the whole puzzle. Note that sometimes a method may only clear some of the notes without filling any cells, causing no visible change. This is most apparent with Bowman's Bingo, which either solves the puzzle or clears 1 note.

In [25]:
import random

puzzleFile = open("puzzleSample.txt")

puzzleNum = random.randrange(1000)

print("Running puzzle #" + str(puzzleNum))
sleep(1.1)
for i in range(puzzleNum):
    puzzleFile.read(100)
puzzleFile.read(13)
puzzleCode = puzzleFile.read(81)

puzzle = SudokuPuzzle(puzzleCode)
puzzle.solve(printUpdates = True, sleepTime = 1.1)
sleep(1.1)
clear_output(wait=True)
print("Completed puzzle #" + str(puzzleNum) + ":")
print(puzzle)
print("Original puzzle #" + str(puzzleNum) + ":")
print(SudokuPuzzle(puzzleCode))

Completed puzzle #945:
895|146|723
347|892|651
126|753|489
-----------
961|487|235
753|269|148
284|531|967
-----------
672|318|594
539|674|812
418|925|376

Original puzzle #945:
890|040|703
007|800|000
006|003|000
-----------
061|000|000
703|209|108
000|000|960
-----------
000|300|500
000|004|800
408|020|076



# Solve a Lot of Puzzles
This next code block runs through all the sample puzzles that I copied into our file (1000 puzzles total)

In [27]:
puzzleFile = open("puzzleSample.txt")

for line in range(1000):
    puzzleFile.read(13)
    puzzle = SudokuPuzzle(puzzleFile.read(81))
    if not puzzle.solve(printUpdates = False, sleepTime = 0):
        raise Exception("Failed to solve puzzle")
    puzzleFile.read(6)

# Solve a Lot of Puzzles (and Print Them)
This code block is just like the one before, except we print the results to screen.

In [26]:
puzzleFile = open("puzzleSample.txt")

for line in range(1000):
    puzzleFile.read(13)
    puzzle = SudokuPuzzle(puzzleFile.read(81))
    print("Original puzzle #" + str(line+1) + ":")
    print(puzzle)
    puzzle.solve(printUpdates = False, sleepTime = 0)
    print("Completed puzzle #" + str(line+1) + ":")
    print(puzzle)
    puzzleFile.read(6)

Original puzzle #1:
020|900|000
048|000|031
000|063|020
-----------
009|407|003
003|080|200
400|105|600
-----------
030|570|000
250|000|180
000|006|050

Completed puzzle #1:
325|941|768
648|752|931
971|863|524
-----------
569|427|813
713|689|245
482|135|679
-----------
136|578|492
257|394|186
894|216|357

Original puzzle #2:
100|800|570
000|009|210
090|040|000
-----------
300|900|050
007|000|300
020|006|008
-----------
000|020|040
071|400|000
064|007|003

Completed puzzle #2:
143|862|579
658|739|214
792|541|836
-----------
316|978|452
987|254|361
425|316|798
-----------
839|625|147
271|493|685
564|187|923

Original puzzle #3:
002|000|800
005|020|100
460|000|029
-----------
130|060|052
009|080|400
000|302|000
-----------
006|070|200
700|000|008
020|519|070

Completed puzzle #3:
312|947|865
985|623|147
467|851|329
-----------
138|764|952
279|185|436
654|392|781
-----------
596|478|213
741|236|598
823|519|674

Original puzzle #4:
802|600|009
000|058|000
006|000|401
-----------
090|406|005

Completed puzzle #36:
419|372|568
857|461|329
263|598|714
-----------
972|184|635
538|726|491
641|935|872
-----------
724|813|956
386|259|147
195|647|283

Original puzzle #37:
036|000|200
000|084|006
100|600|004
-----------
010|090|300
050|406|020
008|070|010
-----------
300|005|001
800|730|000
009|000|870

Completed puzzle #37:
436|951|287
297|384|156
185|627|934
-----------
714|298|365
953|416|728
628|573|419
-----------
372|845|691
861|739|542
549|162|873

Original puzzle #38:
920|504|038
000|603|000
000|070|000
-----------
105|000|806
400|108|002
007|060|500
-----------
002|000|900
740|000|051
300|020|007

Completed puzzle #38:
926|514|738
874|693|125
513|872|469
-----------
135|247|896
469|158|372
287|369|514
-----------
652|781|943
748|936|251
391|425|687

Original puzzle #39:
000|000|000
418|060|030
270|830|000
-----------
006|002|805
300|050|007
801|900|300
-----------
000|094|063
040|080|791
000|000|000

Completed puzzle #39:
563|429|178
418|765|239
279|831|654
-----------
796

Completed puzzle #74:
345|716|892
912|548|376
768|923|541
-----------
423|675|918
876|139|425
591|482|763
-----------
259|364|187
637|891|254
184|257|639

Original puzzle #75:
009|300|100
000|600|007
046|007|002
-----------
090|000|081
000|010|000
680|000|090
-----------
500|200|710
400|001|000
007|004|800

Completed puzzle #75:
279|345|168
135|682|947
846|197|532
-----------
792|456|381
354|918|276
681|723|495
-----------
568|239|714
423|871|659
917|564|823

Original puzzle #76:
080|000|050
700|300|608
029|050|100
-----------
000|407|080
003|090|200
060|502|000
-----------
008|040|510
904|005|003
070|000|040

Completed puzzle #76:
381|624|759
745|319|628
629|758|134
-----------
192|467|385
453|891|276
867|532|491
-----------
238|946|517
914|275|863
576|183|942

Original puzzle #77:
003|000|051
006|054|020
040|309|000
-----------
017|000|040
600|000|009
050|000|370
-----------
000|702|030
030|810|700
870|000|600

Completed puzzle #77:
293|687|451
786|154|923
145|329|867
-----------
317

Completed puzzle #109:
784|619|523
613|524|798
295|738|416
-----------
479|265|381
152|893|647
836|471|952
-----------
921|357|864
368|942|175
547|186|239

Original puzzle #110:
006|070|021
800|000|450
000|005|600
-----------
030|008|075
000|706|000
790|100|060
-----------
004|200|000
057|000|003
260|030|700

Completed puzzle #110:
546|879|321
871|362|459
923|415|687
-----------
632|948|175
418|756|932
795|123|864
-----------
384|297|516
157|684|293
269|531|748

Original puzzle #111:
830|000|047
006|301|900
000|000|000
-----------
007|508|400
900|020|005
350|000|016
-----------
460|209|071
000|070|000
028|000|590

Completed puzzle #111:
831|952|647
576|341|982
249|867|153
-----------
617|538|429
984|126|735
352|794|816
-----------
465|289|371
193|475|268
728|613|594

Original puzzle #112:
083|000|450
020|050|090
600|982|003
-----------
000|030|000
001|405|300
300|627|004
-----------
000|000|000
508|000|901
060|000|030

Completed puzzle #112:
983|761|452
127|354|698
654|982|713
--------

Completed puzzle #143:
239|184|657
641|795|382
857|632|149
-----------
416|327|895
783|951|426
592|468|713
-----------
128|573|964
364|219|578
975|846|231

Original puzzle #144:
070|420|300
004|000|107
810|000|040
-----------
000|607|004
700|030|006
300|804|000
-----------
050|000|083
408|000|900
007|089|050

Completed puzzle #144:
576|421|398
294|358|167
813|976|245
-----------
182|697|534
749|235|816
365|814|729
-----------
951|742|683
428|563|971
637|189|452

Original puzzle #145:
000|720|053
000|005|900
000|000|008
-----------
040|390|060
300|080|005
010|052|040
-----------
800|000|000
004|800|000
290|034|000

Completed puzzle #145:
469|728|153
783|165|924
125|943|678
-----------
542|397|861
376|481|295
918|652|347
-----------
857|216|439
634|879|512
291|534|786

Original puzzle #146:
030|000|050
009|807|003
400|010|008
-----------
040|230|000
001|000|900
000|064|080
-----------
500|040|007
700|903|800
080|000|040

Completed puzzle #146:
238|496|751
169|857|423
457|312|698
--------

...

Completed puzzle #843:
129|856|437
634|217|958
785|439|612
-----------
518|374|296
942|685|173
367|192|584
-----------
896|523|741
471|968|325
253|741|869

Original puzzle #844:
430|600|001
009|804|503
080|000|040
-----------
090|000|038
000|000|000
720|000|060
-----------
070|000|080
908|107|400
200|006|079

Completed puzzle #844:
437|652|891
619|874|523
582|391|647
-----------
194|765|238
856|923|714
723|418|965
-----------
375|249|186
968|137|452
241|586|379

Original puzzle #845:
400|012|007
000|053|000
010|006|040
-----------
520|000|093
090|040|050
180|000|062
-----------
050|800|070
000|360|000
600|190|008

Completed puzzle #845:
435|912|687
768|453|219
219|786|345
-----------
524|671|893
396|248|751
187|539|462
-----------
953|824|176
871|365|924
642|197|538

Original puzzle #846:
080|009|060
009|100|007
000|076|900
-----------
947|000|036
000|000|000
820|000|794
-----------
004|790|000
600|002|100
070|300|040

Completed puzzle #846:
782|539|461
569|124|387
413|876|925
--------

Completed puzzle #880:
893|516|247
516|427|983
427|398|651
-----------
389|142|765
651|873|429
742|659|318
-----------
964|781|532
175|234|896
238|965|174

Original puzzle #881:
650|090|008
028|000|036
000|005|090
-----------
001|209|000
500|000|003
000|403|100
-----------
070|500|000
390|000|780
800|070|045

Completed puzzle #881:
657|392|418
928|147|536
413|865|297
-----------
731|289|654
549|716|823
286|453|179
-----------
174|538|962
395|624|781
862|971|345

Original puzzle #882:
005|001|000
080|700|002
007|830|050
-----------
524|007|000
000|000|000
000|900|523
-----------
030|072|900
800|009|030
000|500|100

Completed puzzle #882:
945|621|378
683|795|412
217|834|659
-----------
524|317|896
369|258|741
178|946|523
-----------
436|172|985
851|469|237
792|583|164

Original puzzle #883:
380|000|000
005|007|008
007|100|560
-----------
000|040|700
100|903|002
008|010|000
-----------
026|004|900
700|800|200
000|000|034

Completed puzzle #883:
381|569|427
265|437|198
497|182|563
--------

Completed puzzle #917:
431|679|258
259|831|674
678|425|931
-----------
367|958|412
894|712|365
125|346|789
-----------
586|293|147
712|564|893
943|187|526

Original puzzle #918:
090|007|201
030|800|000
004|090|000
-----------
040|360|007
007|000|500
800|045|060
-----------
000|010|900
000|004|010
603|500|070

Completed puzzle #918:
596|437|281
132|856|794
784|192|635
-----------
945|368|127
367|921|548
821|745|369
-----------
478|213|956
259|674|813
613|589|472

Original puzzle #919:
090|006|000
000|847|009
004|100|200
-----------
470|308|690
060|000|040
035|904|078
-----------
006|001|900
800|439|000
000|600|020

Completed puzzle #919:
197|526|483
623|847|519
584|193|267
-----------
471|358|692
968|712|345
235|964|178
-----------
756|281|934
812|439|756
349|675|821

Original puzzle #920:
071|902|040
004|000|100
200|001|007
-----------
000|080|006
006|040|900
100|030|000
-----------
500|600|009
007|000|800
020|807|650

Completed puzzle #920:
371|962|548
954|378|162
268|451|397
--------

Completed puzzle #945:
269|514|837
175|839|462
348|672|195
-----------
583|796|214
691|248|753
427|153|689
-----------
752|361|948
814|925|376
936|487|521

Original puzzle #946:
890|040|703
007|800|000
006|003|000
-----------
061|000|000
703|209|108
000|000|960
-----------
000|300|500
000|004|800
408|020|076

Completed puzzle #946:
895|146|723
347|892|651
126|753|489
-----------
961|487|235
753|269|148
284|531|967
-----------
672|318|594
539|674|812
418|925|376

Original puzzle #947:
042|030|860
600|000|009
003|602|400
-----------
008|105|900
000|040|000
470|000|018
-----------
200|010|004
000|857|000
010|000|090

Completed puzzle #947:
742|931|865
681|574|239
593|682|471
-----------
328|165|947
169|748|352
475|293|618
-----------
256|319|784
934|857|126
817|426|593

Original puzzle #948:
260|010|087
007|602|900
000|000|000
-----------
000|060|000
070|020|050
080|945|010
-----------
050|807|020
006|000|100
709|000|508

Completed puzzle #948:
263|519|487
417|682|935
598|473|261
--------

Completed puzzle #987:
321|678|594
648|951|372
759|432|816
-----------
284|169|753
935|847|261
176|325|489
-----------
412|586|937
893|714|625
567|293|148

Original puzzle #988:
900|001|002
500|000|040
087|090|030
-----------
000|203|090
800|000|003
050|107|000
-----------
040|050|210
030|000|009
100|400|007

Completed puzzle #988:
964|831|572
513|672|948
287|594|631
-----------
476|283|195
821|945|763
359|167|824
-----------
748|359|216
632|718|459
195|426|387

Original puzzle #989:
090|801|040
037|604|890
000|000|000
-----------
020|000|060
054|020|930
000|080|000
-----------
500|000|009
309|040|205
000|030|000

Completed puzzle #989:
295|871|643
137|654|892
486|392|157
-----------
923|415|768
854|726|931
671|983|524
-----------
548|267|319
369|148|275
712|539|486

Original puzzle #990:
006|900|080
700|050|400
090|006|005
-----------
009|030|002
070|102|030
500|080|600
-----------
100|500|020
005|010|007
030|009|100

Completed puzzle #990:
356|974|281
782|351|496
491|826|375
--------